# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.34.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

This dataset contains numerical input variables that come from and PCA algorithm. Due to confidential concerns, the original variables are not available. Features V1, V2, ..., V8 are the outputs of the principal components from the PCA algorithm. The time and amount are the variables that are not transformed by PCA. The target variable is the feature 'Class' and it takes values 1 in case of fraud and 0 otherwise.


This dataset contains transations made by credit cards in September 2012 by European cardholders. This dataset contains 492 frauds out of 284807 transations, and it is highly unbalanced, the positive class (fraud) account for 0.172% of all the transations.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'telco-customer-churn'
experiment=Experiment(ws, experiment_name)

# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "Customer Churn"
description_text = "Customer Churn DataSet for Udacity Capstone Project"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/srees1988/predict-churn-py/main/customer_churn_data.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges
count,7043.000000,7043.000000,7043.000000,7032.000000
mean,0.162147,32.371149,64.761692,2283.300441
std,0.368612,24.559481,30.090047,2266.771362
min,0.000000,0.000000,18.250000,18.800000
25%,0.000000,9.000000,35.500000,401.450000
50%,0.000000,29.000000,70.350000,1397.475000
75%,0.000000,55.000000,89.850000,3794.737500
max,1.000000,72.000000,118.750000,8684.800000


Check the first five rows:

In [3]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,True,False,1,False,No phone service,DSL,No,...,No,No,No,No,Month-to-month,True,Electronic check,29.85,29.85,False
1,5575-GNVDE,Male,0,False,False,34,True,No,DSL,Yes,...,Yes,No,No,No,One year,False,Mailed check,56.95,1889.50,False
2,3668-QPYBK,Male,0,False,False,2,True,No,DSL,Yes,...,No,No,No,No,Month-to-month,True,Mailed check,53.85,108.15,True
3,7795-CFOCW,Male,0,False,False,45,False,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,False,Bank transfer (automatic),42.30,1840.75,False
4,9237-HQITU,Female,0,False,False,2,True,No,Fiber optic,No,...,No,No,No,No,Month-to-month,True,Electronic check,70.70,151.65,True


The column customerID should be removed because they have unique values in the whole column:

In [4]:
df['customerID'].nunique() == df.shape[0]

True

In [5]:
df.drop('customerID', axis=1, inplace=True)
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,True,False,1,False,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,True,Electronic check,29.85,29.85,False
1,Male,0,False,False,34,True,No,DSL,Yes,No,Yes,No,No,No,One year,False,Mailed check,56.95,1889.50,False
2,Male,0,False,False,2,True,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,True,Mailed check,53.85,108.15,True
3,Male,0,False,False,45,False,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,False,Bank transfer (automatic),42.30,1840.75,False
4,Female,0,False,False,2,True,No,Fiber optic,No,No,No,No,No,No,Month-to-month,True,Electronic check,70.70,151.65,True


Check the data types present along the columns:

In [6]:
df.dtypes

gender               object
SeniorCitizen         int64
Partner                bool
Dependents             bool
tenure                int64
PhoneService           bool
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling       bool
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                  bool
dtype: object

Let's check the missing values:

In [7]:
# check missing values
df.isnull().sum()

gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

Let's check the unique values of the float columns:

In [8]:
float_columns = df.select_dtypes(include=['float64']).columns
print(float_columns)

Index(['MonthlyCharges', 'TotalCharges'], dtype='object')


In [9]:
for column in float_columns:
    print(df[column].value_counts())
    print('\n')

20.05     61
19.85     45
19.95     44
19.90     44
20.00     43
          ..
114.75     1
103.60     1
113.40     1
57.65      1
113.30     1
Name: MonthlyCharges, Length: 1585, dtype: int64


20.20      11
19.75       9
19.65       8
20.05       8
19.90       8
           ..
1066.15     1
249.95      1
8333.95     1
7171.70     1
1024.00     1
Name: TotalCharges, Length: 6530, dtype: int64




Let's check the unique values of the float columns:

In [10]:
bool_columns = df.select_dtypes(include=['bool']).columns
print(bool_columns)

Index(['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling', 'Churn'], dtype='object')


In [11]:
for column in bool_columns:
    print(df[column].value_counts())
    print('\n')

False    3641
True     3402
Name: Partner, dtype: int64


False    4933
True     2110
Name: Dependents, dtype: int64


True     6361
False     682
Name: PhoneService, dtype: int64


True     4171
False    2872
Name: PaperlessBilling, dtype: int64


False    5174
True     1869
Name: Churn, dtype: int64




In [12]:
5174/1864

2.7757510729613735

The variable Churn is highly skewed toward False by a factor of 2.77

Let's check the unique values of the object columns:

In [13]:
object_columns = df.select_dtypes(include=['object']).columns
print(object_columns)

Index(['gender', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaymentMethod'],
      dtype='object')


In [14]:
for column in object_columns:
    print(df[column].value_counts())
    print('\n')

Male      3555
Female    3488
Name: gender, dtype: int64


No                  3390
Yes                 2971
No phone service     682
Name: MultipleLines, dtype: int64


Fiber optic    3096
DSL            2421
No             1526
Name: InternetService, dtype: int64


No                     3498
Yes                    2019
No internet service    1526
Name: OnlineSecurity, dtype: int64


No                     3088
Yes                    2429
No internet service    1526
Name: OnlineBackup, dtype: int64


No                     3095
Yes                    2422
No internet service    1526
Name: DeviceProtection, dtype: int64


No                     3473
Yes                    2044
No internet service    1526
Name: TechSupport, dtype: int64


No                     2810
Yes                    2707
No internet service    1526
Name: StreamingTV, dtype: int64


No                     2785
Yes                    2732
No internet service    1526
Name: StreamingMovies, dtype: int64


Month-to-mo

## Train Test Splitting

In [47]:
from sklearn.model_selection import train_test_split
from azureml.data.dataset_factory import TabularDatasetFactory
description_text = "Train and Test splitting from Customer Churn DataSet for Udacity Capstone Project"

churn = df['Churn']

# Split data into train and test data taking into account the variable Churn is highly skewed:
train_dataset, test_dataset = train_test_split(df, test_size=0.2, stratify=churn, random_state=42)

directory = 'train'
if not os.path.exists(directory):
    os.makedirs(directory)
    
directory = 'test'
if not os.path.exists(directory):
    os.makedirs(directory)

# Export data as csv
train_dataset.to_csv("train_data.csv", index=False)
test_dataset.to_csv("test_data.csv", index=False)

# # Upload data to the datastore
datastore = ws.get_default_datastore()
datastore.upload(src_dir='./train', target_path = experiment_name)
datastore.upload(src_dir='./test', target_path = experiment_name)
print('Data uploaded to DataStore')

csv_path_train = [(datastore, experiment_name+'/train_data.csv')]
csv_path_test = [(datastore, experiment_name+'/test_data.csv')]

train_data = Dataset.Tabular.from_delimited_files(path=csv_path_train)
test_data = Dataset.Tabular.from_delimited_files(path=csv_path_test)

display(train_data.to_pandas_dataframe().head())
display(test_data.to_pandas_dataframe().head())

Uploading an estimated of 1 files
Target already exists. Skipping upload for telco-customer-churn/train_data.csv
Uploaded 0 files
Uploading an estimated of 1 files
Target already exists. Skipping upload for telco-customer-churn/test_data.csv
Uploaded 0 files
Data uploaded to DataStore


,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Male,0,False,False,35,False,No phone service,DSL,No,No,Yes,No,Yes,Yes,Month-to-month,False,Electronic check,49.20,1701.65,False
1,Male,0,True,True,15,True,No,Fiber optic,Yes,No,No,No,No,No,Month-to-month,False,Mailed check,75.10,1151.55,False
2,Male,0,True,True,13,False,No phone service,DSL,Yes,Yes,No,Yes,No,No,Two year,False,Mailed check,40.55,590.35,False
3,Female,0,True,False,26,True,No,DSL,No,Yes,Yes,No,Yes,Yes,Two year,True,Credit card (automatic),73.50,1905.70,False
4,Male,0,True,True,1,True,No,DSL,No,No,No,No,No,No,Month-to-month,False,Electronic check,44.55,44.55,False


,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Male,0,True,True,72,True,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Two year,True,Credit card (automatic),114.05,8468.20,False
1,Female,1,False,False,8,True,Yes,Fiber optic,No,No,No,Yes,Yes,Yes,Month-to-month,True,Credit card (automatic),100.15,908.55,False
2,Female,0,True,True,41,True,Yes,DSL,Yes,Yes,Yes,No,Yes,No,One year,True,Credit card (automatic),78.35,3211.20,False
3,Male,0,True,False,18,True,No,Fiber optic,No,No,Yes,Yes,No,No,Month-to-month,False,Electronic check,78.20,1468.75,False
4,Female,0,True,False,72,True,Yes,DSL,Yes,Yes,Yes,No,Yes,Yes,Two year,True,Credit card (automatic),82.65,5919.35,False


## Cluster Provisioning


In [48]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

cluster_name = "cluster-vhcg"
# verify that the cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name = cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', max_nodes = 4, idle_seconds_before_scaledown=120)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [49]:
# TODO: Put your automl config here
automl_config = AutoMLConfig(
                                compute_target=cluster_name,
                                task='classification',
                                training_data=train_data,
                                test_data = test_data,
                                label_column_name='Churn',
                                # n_cross_validations=10,
                                # validation_size=0.2,
                                primary_metric='AUC_weighted',
                                experiment_timeout_minutes=30,
                                max_concurrent_iterations=5,
                                max_cores_per_iteration=-1, 
                                featurization= 'auto',
                                debug_log = "automl_errors.log",                                

                                )

##  Create AutoML Pipeline

In [ ]:

from azureml.pipeline.core import PipelineData, TrainingOutput

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

Create AutoMLStep

In [ ]:
automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)

In [ ]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step])

In [ ]:
pipeline_run = experiment.submit(pipeline)

In [ ]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

In [ ]:
pipeline_run.wait_for_completion()

## Examine Results

Retrieve the metrics of all child runs

In [ ]:
metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)

In [ ]:
import json
with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df


Retrieve the Best Model

In [ ]:

# Retrieve best model from Pipeline Run
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

In [ ]:
import pickle

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
best_model

In [ ]:
best_model.steps

## Test the model

Load Test Data

In [ ]:
dataset_test = test_data
df_test = dataset_test.to_pandas_dataframe()
df_test = df_test[pd.notnull(df_test['Churn'])]

y_test = df_test['Churn']
X_test = df_test.drop(['Churn'], axis=1)

Testing Our Best Fitted Model

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
ypred = best_model.predict(X_test)
cm = confusion_matrix(y_test, ypred)
accuracy = accuracy_score(y_test, ypred)
auc = roc_auc_score(y_test, ypred)


In [ ]:
# Visualize the confusion matrix
pd.DataFrame(cm).style.background_gradient(cmap='Blues', low=0, high=0.9)

In [ ]:
print("Accuracy score is: ", accuracy)
print("AUC score is: ", auc)


## Publish and run from REST endpoint

In [ ]:

published_pipeline = pipeline_run.publish_pipeline(
    name="Bankmarketing Train", description="Training bankmarketing pipeline", version="1.0")

published_pipeline

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()

In [ ]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "pipeline-rest-endpoint"}
                        )

In [ ]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception("Received bad response from the endpoint: {}\n"
                    "Response Code: {}\n"
                    "Headers: {}\n"
                    "Content: {}".format(rest_endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)

In [ ]:

from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails

published_pipeline_run = PipelineRun(ws.experiments["pipeline-rest-endpoint"], run_id)
RunDetails(published_pipeline_run).show()